In [1]:
# %pip install opencv-python
%pip install imageio[ffmpeg]
%pip install tqdm

In [2]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/gdrive')
# Define base folder path
base_path = '/content/gdrive/MyDrive/RGB_data_stream'

Mounted at /content/gdrive


In [3]:
import os

csv_path = os.path.join(base_path,'data/3DYoga90_corrected.csv')
sequence_path = os.path.join(base_path, 'short/downloaded_log.txt')
save_path = os.path.join(base_path, 'Second_Resnet_nov6')
os.makedirs(save_path, exist_ok=True)
pose_list = ['mountain', 'half-way-lift', 'standing-forward-bend', 'downward-dog']
NUM_CLASSES = len(pose_list)
video_dir = os.path.join(base_path, 'short')
preprocessed_dir = os.path.join(base_path, 'complete_pre_processed')
os.makedirs(preprocessed_dir, exist_ok=True)

In [4]:
# Constants
FRAME_HEIGHT = 224  # VGG16 input size
FRAME_WIDTH = 224
SEQUENCE_LENGTH = 5

In [5]:
import os
import torch
import pandas as pd
import numpy as np
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import imageio

class YogaVideoDataset(Dataset):
    def __init__(self, csv_path, sequence_path, pose_list, video_dir, preprocessed_dir):
        with open(sequence_path) as f:
            sequence_list = f.read().splitlines()
            sequence_list = [int(x) for x in sequence_list]

        self.df = pd.read_csv(csv_path)
        self.df = self.df[self.df['sequence_id'].isin(sequence_list)]
        self.df = self.df[self.df['l3_pose'].isin(pose_list)]

        self.pose_to_idx = {pose: idx for idx, pose in enumerate(pose_list)}
        self.length_of_dataset = len(self.df)

        self.video_dir = video_dir
        self.preprocessed_dir = preprocessed_dir

        os.makedirs(self.preprocessed_dir, exist_ok=True)

        self.transforms = transforms.Compose([
            transforms.Resize((FRAME_HEIGHT, FRAME_WIDTH)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.cache = dict()

    def __len__(self):
        return self.length_of_dataset

    def __getitem__(self, i):
        sequence_id = self.df.iloc[i]['sequence_id']
        pose = self.df.iloc[i]['l3_pose']
        label = self.pose_to_idx[pose]

        # Check cache first
        if sequence_id in self.cache:
            frames = self.cache[sequence_id]
        else:
            # Path to preprocessed tensor file
            preprocessed_path = os.path.join(self.preprocessed_dir, f"{sequence_id}.pt")

            if os.path.exists(preprocessed_path):
                frames = torch.load(preprocessed_path, weights_only=True)
            else:
                # Process and save if not already preprocessed
                video_path = os.path.join(self.video_dir, f"{sequence_id}.mp4")
                frames = self._get_frames(video_path)
                torch.save(frames, preprocessed_path)

            # Add to cache
            # self.cache[sequence_id] = frames

        return frames, label

    def _get_frames(self, video_path):
        reader = imageio.get_reader(video_path, 'ffmpeg')
        fps = imageio.get_reader(video_path, 'ffmpeg').get_meta_data()['fps'] // 1
        total_frames = reader.count_frames()

        # print(fps)
        # print(total_frames)
        # Sample SEQUENCE_LENGTH = 7 frames per second
        # indices = np.linspace(0, fps - 1, SEQUENCE_LENGTH, dtype=int)
        print(video_path)
        indices = np.arange(0, fps - 1, fps // SEQUENCE_LENGTH, dtype=int)

        # print(indices)
        frames = []
        for i, frame in enumerate(reader):
            if i % fps in indices:
                frame = Image.fromarray(frame)
                frame = self.transforms(frame)
                frames.append(frame)
                # print('Frame number', i % fps, 'Frame', i)

        reader.close()
        # print('Frame Length', len(frames))
        frames = torch.stack(frames)
        return frames

In [6]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    frames, labels = zip(*batch)

    # Pad the frames to the maximum sequence length in the batch
    frames_padded = pad_sequence(frames, batch_first=True, padding_value=0)  # Shape: [batch_size, max_seq_len, 3, 224, 224]

    labels = torch.tensor(labels)

    return frames_padded, labels

In [11]:
from tqdm import tqdm
from torch.utils.data import DataLoader

dataset = YogaVideoDataset(csv_path, sequence_path, pose_list, video_dir, preprocessed_dir)
loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    collate_fn=collate_fn  # Use custom collate function
)
train_loader_tqdm = tqdm(enumerate(loader), total=len(loader), desc="Processing Clips")
for batch_idx, (inputs, labels) in train_loader_tqdm:
          print("DONE")

Processing Clips:   0%|          | 0/22 [00:00<?, ?it/s]

/content/gdrive/MyDrive/RGB_data_stream/short/1776.mp4
/content/gdrive/MyDrive/RGB_data_stream/short/2029.mp4
/content/gdrive/MyDrive/RGB_data_stream/short/1552.mp4
/content/gdrive/MyDrive/RGB_data_stream/short/2038.mp4


Processing Clips:   0%|          | 0/22 [00:20<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 6653) exited unexpectedly

In [12]:
for i, data in enumerate(dataset):
  print('Data', i)

Data 0
/content/gdrive/MyDrive/RGB_data_stream/short/1002.mp4
Data 1
Data 2
Data 3
Data 4
/content/gdrive/MyDrive/RGB_data_stream/short/1006.mp4
Data 5
Data 6
Data 7
Data 8
Data 9
Data 10
Data 11
/content/gdrive/MyDrive/RGB_data_stream/short/1013.mp4
Data 12
Data 13
Data 14
Data 15
Data 16
Data 17
Data 18
Data 19
Data 20
Data 21
Data 22
/content/gdrive/MyDrive/RGB_data_stream/short/1025.mp4
Data 23
Data 24
Data 25
Data 26
Data 27
Data 28
Data 29
/content/gdrive/MyDrive/RGB_data_stream/short/1032.mp4
Data 30
Data 31
Data 32
Data 33
Data 34
Data 35
Data 36
Data 37
Data 38
Data 39
Data 40
Data 41
Data 42
Data 43
Data 44
Data 45
Data 46
/content/gdrive/MyDrive/RGB_data_stream/short/1049.mp4
Data 47
Data 48
Data 49
Data 50
Data 51
/content/gdrive/MyDrive/RGB_data_stream/short/1056.mp4
Data 52
Data 53
Data 54
Data 55
Data 56
Data 57
Data 58
Data 59
Data 60
Data 61
Data 62
Data 63
Data 64
Data 65
Data 66
Data 67
Data 68
Data 69
Data 70
Data 71
Data 72
/content/gdrive/MyDrive/RGB_data_stream/s